# Predictive Maintenance Dashboard

This dashboard visualizes:
- ⚠️ Detected anomalies
- 💡 Root-cause suggestions via GenAI + RAG
- 📊 Metrics per machine & over time

Use the filters to drill into specific machines, time windows, or severity levels.


In [0]:
# Load Dependencies & Data

from pyspark.sql.functions import col
import pyspark.sql.functions as F

df = spark.table("sensor_data_test_scored")

display(df.limit(5))

In [0]:

# Machine ID Filter (Widget)
dbutils.widgets.dropdown("machine_id", "All", ["All"] + [str(x) for x in range(1,6)], "Machine ID")

selected_machine = dbutils.widgets.get("machine_id")
if selected_machine == "All":
    df_filtered = df
else:
    df_filtered = df.filter(col("machine_id") == int(selected_machine)).filter(col("anomaly") == -1)

In [0]:
# Time Range Filter (Widget)

dbutils.widgets.text("start_date", "2025-07-13", "Start Date (YYYY-MM-DD)")
dbutils.widgets.text("end_date", "2025-07-14", "End Date (YYYY-MM-DD)")

start = dbutils.widgets.get("start_date")
end = dbutils.widgets.get("end_date")

df_filtered = df_filtered.filter((col("timestamp") >= start) & (col("timestamp") <= end))

In [0]:
# Anomaly Table (Interactive)
display(df_filtered.orderBy(col("timestamp").desc()))

In [0]:
# Anomalies Over Time (Line Chart)

df_time = df_filtered.groupBy(F.window("timestamp", "1 hour")).count().withColumnRenamed("count", "anomaly_count")
display(df_time)

Databricks visualization. Run in Databricks to view.

In [0]:
# Anomalies per Machine (Bar Chart)

df_machine = df_filtered.groupBy("machine_id").count().withColumnRenamed("count", "anomaly_count")
display(df_machine)

Databricks visualization. Run in Databricks to view.